## Lab 3 Part 2 - EECE.4860/5860 at UMass Lowell

Complete the missing code to create a new tool based on RAG, and execute a complex query that invokes the tools.

This example uses LangGraph APIs.

We will use ChatNVIDIA model because it supports local mode when working with LangGraph. You need to apply for a free API key via https://build.nvidia.com

In [ ]:
import sys
import os

%pip install --upgrade langchain langgraph langchain-community langchain-huggingface langchain-nvidia-ai-endpoints wikipedia numexpr pypdf sentence-transformers

In [ ]:
from langchain_core.tools import tool

@tool
def magic_function(input: int) -> int:
    """Applies a magic function to an input."""
    return input + 2

tools = [magic_function]

query = "what is the value of magic_function(3)?"

In [ ]:
from typing import Annotated
from langchain_core.tools import tool
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.indexes import VectorstoreIndexCreator
from langchain_huggingface import HuggingFaceEmbeddings

top_k = 1
max_tokens = 512
chunk_size=1000
overlap=50

# The new tool you need to create is a RAG tool that answer queries about DeepSeek-R1
# The tool leverages a few APIs that you see in the simple_rag.ipynb example, including
# RecursiveCharacterTextSplitter(), VectorstoreIndexCreator(), vectorstore.similarity_search()
#
loader = PyPDFLoader("https://arxiv.org/pdf/2501.12948.pdf")
docs = loader.load()
# TODO: call  RecursiveCharacterTextSplitter() using chunk_size and overlap
# text_splitter = ...

# TODO: split the docuement using text_splitter and docs
# splits = ...

# TODO: create the vector database by calling VectorstoreIndexCreator()
# TODO: using HuggingFaceEmbedding() and "intfloat/multilingual-e5-large-instruct" model
#index = VectorstoreIndexCreator(
#    embedding= ...,
#    text_splitter=...
#).from_loaders([loader])


@tool
def query_deepseek_r1_paper(input: Annotated[str, "Query relating to DeepSeek-R1"]) -> str:
    """Provides context that can be used to answer questions on DeepSeek-R1"""
    
    print(f"Received input query: {input}\n")

    # TODO: perform similarity search on the input and top_k
    # results = ...
    
    if not results:
        print("No results found for the query.\n\n")
        return "no results"
    
    # TODO: concatenate the results using "\n" as a separator
    # context = 

    if context:
        print(f"Context found:\n{context}\n\n")
        
    return context
 

In [ ]:
#use ChatNVIDIA model because it supports local mode when working with LangGraph.
#You need to apply for a free API key via https://build.nvidia.com
import os
import getpass

if not os.getenv("NVIDIA_API_KEY"):
    # Note: the API key should start with "nvapi-"
    os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter your NVIDIA API key: ")

from langchain_nvidia_ai_endpoints import ChatNVIDIA

#print(ChatNVIDIA.get_available_models())
# not all the models in ChatNVIDIA supports tools.
tool_models = [
    model for model in ChatNVIDIA.get_available_models() if model.supports_tools
]
print(tool_models)

llm_nvidia = ChatNVIDIA(model="meta/llama-3.1-405b-instruct",
                           temperature=0.1,
                            max_tokens=512,
                            top_p=1.0,)

In [ ]:
from langchain.chains import LLMMathChain, LLMChain
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType

wikipedia_wrapper = WikipediaAPIWrapper()
wikipedia_tool = Tool(
    name="Wikipedia",
    func=wikipedia_wrapper.run,
    description="A tool for searching the Internet to find various information on the topics mentioned."
)
math_chain = LLMMathChain.from_llm(llm=llm_nvidia, verbose=True)
calculator = Tool.from_function(
    name="Calculator",
    func=math_chain.run,
    description="A tool for solving mathematical problems. Provide only the mathematical expressions."
)

tools=[wikipedia_tool, calculator, magic_function, query_deepseek_r1_paper]

In [ ]:
from langgraph.prebuilt import create_react_agent

langgraph_agent_executor = create_react_agent(llm_nvidia, tools)

# Increase the recursion limit in the configuration
config = {
   "recursion_limit": 20  # Increase the limit as needed
}

query1 = "In what year was the film Departed with Leopnardo Dicaprio released? What is this year raised to the power of 0.43?"

query2 = "Use the year the film Departed with Leopnardo Dicaprio was released to calculate the value of magic_function."

query3 = "What are the main limitations of DeepSeek-R1?"

messages1 = langgraph_agent_executor.invoke({"messages": [("human", query1)]}, config=config)

print('-'*200)
print("Query 1:", query1)
print('-'*200)
print("Response 1:", messages1["messages"][-1].content)
print('-'*200, '\n')

messages2 = langgraph_agent_executor.invoke({"messages": [("human", query2)]}, config=config)
print('-'*200)
print("Query 2:", query2)
print('-'*200)
print("Response 2:", messages2["messages"][-1].content)
print('-'*200, '\n')


messages3 = langgraph_agent_executor.invoke({"messages": [("human", query3)]}, config=config)
print('-'*200)
print("Query 3:", query3)
print('-'*200)
print("Response 3:", messages3["messages"][-1].content)
print('-'*200, '\n')